<a href="https://colab.research.google.com/github/Sidy3143/llm-projects/blob/main/PDF_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q fsspec==2025.3.0 gcsfs transformers accelerate peft bitsandbytes datasets trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="bfloat16",
    bnb_double_quant=True,
)

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `mistral` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `mistral`


In [ ]:
model_name="mistralai/Mistral-7B-Instruct-v0.2"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"":0},
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
from google.colab import files
uploaded = files.upload()

Saving Test Time Scaling Paper.pdf to Test Time Scaling Paper.pdf


In [9]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.5 MB/s eta 0:00:00


In [10]:
import PyPDF2

def load_pdf(file_name):
  with open(file_name, 'rb') as f:
    reader = PyPDF2.PdfReader(f)
    text = ''
    for page in reader.pages:
      text += page.extract_text() + '\n'

  return text

In [11]:
file_name = list(uploaded.keys())[0]
document_text = load_pdf(file_name)

In [12]:
!pip -q install llama-index
!pip -q install llama-index-embeddings-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 9.8 MB/s eta 0:00:00


In [13]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.schema import Document
from llama_index.core.postprocessor import SimilarityPostprocessor

In [ ]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Settings.llm = None
Settings.chunk_size = 512
Settings.chunk_overlap = 25

In [16]:
document_text

'2024-8-7\nScaling LLM Test-Time Compute Optimally can\nbe More Effective than Scaling Model Parameters\nCharlie Snell♦, 1, Jaehoon Lee2, Kelvin Xu♣, 2and Aviral Kumar♣, 2\n♣Equal advising,1UC Berkeley,2Google DeepMind,♦Work done during an internship at Google DeepMind\nEnabling LLMs to improve their outputs by using more test-time computation is a critical step towards\nbuilding generally self-improving agents that can operate on open-ended natural language. In this paper,\nwe study the scaling of inference-time computation in LLMs, with a focus on answering the question: if an\nLLM is allowed to use a fixed but non-trivial amount of inference-time compute, how much can it improve its\nperformance on a challenging prompt? Answering this question has implications not only on the achievable\nperformance of LLMs, but also on the future of LLM pretraining and how one should tradeoff inference-time\nand pre-training compute. Despite its importance, little research attempted to understand t

In [15]:
documents = [Document(text=document_text)]

In [17]:
index = VectorStoreIndex.from_documents(documents)

In [18]:
top_k = 3

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k
)

In [19]:
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)]
    )

In [20]:
query = "What is test time scaling about?"

response = query_engine.query(query)

In [22]:
context = "Context:\n"
for i in range(top_k):
  context += response.source_nodes[i].text

In [23]:
context

'Context:\nby training a process-based dense verifier [ 22,45] and searching against this\nverifier), the ability scale test-time compute could be greatly improved, as we show in the paper.\nTo understand the benefits of scaling up test-time computation, we carry out experiments on the\nchallenging MATH [ 13] benchmark using PaLM-2 [ 3] models specifically fine-tuned1to either revise\n1Capability-specific finetuning is necessary to induce revision and verification capabilities into the base model on MATH\n2\nincorrect answers [28] (e.g. improving the proposal distribution; Section 6) or verify the correctness of\nindividual steps in an answer using a process-based reward model (PRM) [ 22,45] (Section 5). With\nboth approaches, we find that the efficacy of a particular test-time compute strategy depends critically\non both the nature of the specific problem at hand and the base LLM used. For example, on easier\nproblems, for which the base LLM can already readily produce reasonable resp

In [24]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [30]:
system_prompt = f"""Your an AI assistant. Your goal is to provide answers given the provided relevant context . Use the context if it is relevant to the question. If you don't know the answer, just say you don't know.
{context}

Please respond to the following question:"""

message_template = [
    {"role":"system", "content": system_prompt},
    {"role":"user", "content": query}
]

input = tokenizer.apply_chat_template(message_template)

input = torch.tensor([input]).to(device)

In [31]:
output = model.generate(input, max_length=4096)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [35]:
decoded = tokenizer.decode(output[0])

In [36]:
start = decoded.find("[/INST]")

decoded[start:]

'[/INST] Test time scaling refers to the process of improving the performance of a language model at test time by utilizing additional computation. This can be achieved through various strategies such as iteratively refining initial answers by predicting a sequence of revisions or sampling new responses independently in parallel or deploying tree-search against a process-based reward model. The specific approach for utilizing test-time compute depends on the nature of the problem and the base language model used. The findings from the research suggest that an adaptive "compute-optimal" strategy is necessary to make the best use of additional computation and improve test-time compute scaling.</s>'